prerequisites, to correctly execute this jupyter notebook you should have an environment with the following packages:

```
# note that conda-forge is used separately
# note that if you have a GPU you should install tensorflow-gpu and cudatoolkit
# in the first conda install line.

conda create -n keras
conda activate keras
conda install tensorflow jupyter numpy matplotlib scikit-image
conda install -c conda-forge nibabel
```

# Lung Segmentation 

In this notebook we will implement a 3D convolutional U-NET which capable of segmenting the lung volume. Here you will find several exercises to better understand what a U-NET is, how to manage the training pipeline and how to evaluate the outcome of a deep neural network.

The UNET is a popular neural network architecture for semantic segmentation tasks, which involve assigning a label to each pixel in an input image or volume. It was first introduced by Olaf Ronneberger, Philipp Fischer, and Thomas Brox in their 2015 paper, "U-Net: Convolutional Networks for Biomedical Image Segmentation".

The UNET architecture consists of two main parts: a contracting path and an expanding path. The contracting path is designed to capture context and extract features from the input image or volume, while the expanding path is designed to generate the final segmentation mask by upsampling the output of the contracting path and fusing it with the corresponding feature maps from the contracting path.

The contracting path is composed of multiple encoder blocks, each consisting of two convolutional layers followed by a max pooling layer. The convolutional layers are used to extract features and the max pooling layers are used to reduce the spatial resolution of the feature maps while increasing the number of feature channels. This helps to capture more global context and abstract features from the input image or volume.

The expanding path is composed of multiple decoder blocks, each consisting of an upsampling layer followed by two convolutional layers. The upsampling layer is used to increase the spatial resolution of the feature maps while reducing the number of feature channels. The output of the upsampling layer is then concatenated with the corresponding feature maps from the contracting path, which are at the same spatial resolution but have a higher number of feature channels. This helps to combine low-level features from the contracting path with high-level features from the expanding path, resulting in a more accurate segmentation mask.

At the end of the expanding path, a 1x1 convolutional layer with a sigmoid activation function is used to generate the final segmentation mask. This layer produces a probability map indicating the likelihood of each pixel belonging to the target class.

One of the key innovations of the UNET architecture is the use of skip connections, which connect the corresponding feature maps from the contracting path to the expanding path. This helps to preserve spatial information and enables the network to recover fine-grained details in the final segmentation mask.

![Unet example](Answer/unetEX.png)

The image shows a representation of a UNet structure [1]. The tipical features of a UNet are:
* Left decoding layers
* bottleneck layer (i.e. lowest part of the network)
* Right encoding layers
* Residual sum from decoding to encoding

Here you will not implement a UNet on your own, but in the next blocks (beneath some comments) there will be some *void* section that you will need to write to make this notebook work. Obviously you may look at the answer, but try to perform the tasks on your own as much as you can.

We start by importing the packages needed for this work.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from glob import glob
from tensorflow import keras

We start by separating validation data from the training data. In this case, with only 60 images available, we will divide the dataset into 2. 90% of the images will be used in training while the remaining 10% will be used as validation samples during the training. The data is availale in the `data/lctsc` directory. At that path you will find some other subdirectories composed by a 5 digits number. Each number represents a clinical case (i.e. patient) and each case is a directory containing a CT image and the corresponding lung segmentation mask. 

```
# ls -l data/lctsc/*
data/lctsc/02219:
total 292
-rw-rw-r-- 1 user user 286100 May 12 16:22 ct.nii.gz
-rw-rw-r-- 1 user user  10120 May 12 16:22 lung.nii.gz
...
```

These images are saved in NIfTI format, so you will need to use the [nibabel](https://nipy.org/nibabel/) library to open them. To see how nibabel works and have some practice you can go back to the 3Dimages notebook.

Now lets go back to our objective -> define test and train dataset. This means you need to define two numpy arrays with 5 dimensions. The first one is the sample dimension (i.e. the index that specifies a certain image, we have 60 images), the following three dimension is the space where the CT scan is defined (i.e. in our specific case 64x64x64) while the last dimension specifies the number of channels (i.e. the ammount of *colors* of the image, 1 for CT images). To define these array follow the next steps:

* list the patients in `data/lctsc` (tip: use glob)
* open the lung ct images with nibabel and store the windowed (1300,-350) image with values from 0 to 1 in the image numpy array (tip: before uploading all the images, define an empty array with the correct dimensions)
* open the lung segmentation images and store the data into the label numpy array (tip: same as above)
* Define the train dataset by taking the first 90% of data from both the preceding arrays
* Define the test dataset by taking the last 10% of the data

**NOTE: don't forget to plot the data to see if you are dooing the correct things**

In [ ]:
def windower(data,wmin,wmax):
    """
    windower function, gets in input a numpy array,
    the minimum HU value and the maximum HU value.
    It returns a windowed numpy array with the same dimension of data
    containing values between 0 and 1
    """
    dump = data.copy()
    dump[dump>=wmax] = wmax
    dump[dump<=wmin] = wmin
    dump -= wmin
    w = wmax - wmin
    return dump / w

patients = glob() # set the coorect path

ct_images  = np.zeros() # set the correct schape
seg_images = np.zeros() # set the correct schape

for i,p in zip(range(len(patients)),patients):
    ct_images[i,...,0]  = windower( HERE ,-1000,300) # Here the image
    seg_images[i,...,0] = nib.load( HERE ).get_fdata() # the mask
    seg_images[i,...,0][seg_images[i,...,0]>0]=1
    
train_img = ct_images[  ] # SELECT first 90% of images
train_seg = seg_images[0:int(0.9*len(patients))] # SELECT first 90% of masks

test_img = ct_images[int(0.9*len(patients)):] # SELECT last 10% of images
test_seg = seg_images[int(0.9*len(patients)):] # SELECT last 10% of masks


# check min and max value of the arrays and check if those are as how you expected them.

In [ ]:
for p,ct,seg in zip(patients,ct_images,seg_images):
    
    # set 2 axes 1 for the image 1 for the mask
    fig,ax=plt.subplots()
    
    im=ax[0].imshow() # pass the ct img as z index mean
    ax[1].imshow() # pass the mask img as z index mean
    for a in ax:
        a.axis("off")
        
    plt.sho()
    pl.close()

It is now time to complicate things and start defining the Unet neural network model. To make things a little bit more easy, there are 2 predefined *layer* functions: `down` and `up`. The first function is the one that decodes the image into features while the latter one encodes the features back into a masked image. The `down` function needs as an input a layer and the number of filters to output after the convolution. While the `up` function needs the same variable with in addition the corresponding layer of the decoding process which will be concatenated to the encoding layer. Both function return a processed layer, the down layer with half the spacial dimensions and k filters as channels, the up layer returns a layer with double the spacial dimensions an k filters as channels.

for example, down:
```
    down(input,8)                 output
  input layer shape         output layer shape
(None, 64, 64, 64, 1)  --> (None, 32, 32, 32, 8)
```

up:
```
    down(input,16)                 output
  input layer shape          output layer shape
(None, 16, 16, 16, 32)  --> (None, 32, 32, 32, 16)
```

The full unerstanding of how the layers are interconnected is out of the scope of this notebook. The important things to remember are:

* maxpooling3D: `input (None, X, Y, Z, l) -> output (None, X/2, Y/2, Z/2, l)`
* upsampling3D: `input (None, X, Y, Z, l) -> output (None, X*2, Y*2, Z*2, l)`
* conv3D(l, padding='same'): `input (None, X, Y, Z, l') -> output (None, X, Y, Z, l)`
* conv3DTranspose(l, padding='same'): `input (None, X, Y, Z, l') -> output (None, X, Y, Z, l)`

Now, try to complete the model by following the tips in the comments. At the end try to display the model by using the `summary` method on the model.


In [ ]:
# down
def down(previous_layer,k, **kargs):
    d = keras.layers.Conv3D(k,3,padding="same",**kargs)(previous_layer)
    d = keras.layers.BatchNormalization()(d)
    d = keras.layers.Activation('relu')(d)
    d = keras.layers.Conv3D(k,3,padding="same",**kargs)(d)
    d = keras.layers.BatchNormalization()(d)
    d = keras.layers.Activation('relu')(d)
    d = keras.layers.Conv3D(k,3,padding="same",**kargs)(d)
    d = keras.layers.BatchNormalization()(d)
    d = keras.layers.MaxPooling3D()(d)

    res = keras.layers.Conv3D(k,3,strides=2,padding="same",**kargs)(previous_layer)
    d = keras.layers.Add()([res,d])
    d = keras.layers.Activation('relu')(d)
    return d

# up
def up(previous_layer,down_layer,k, **kargs):
    u = keras.layers.Conv3DTranspose(k,3,padding="same",**kargs)(previous_layer)
    u = keras.layers.BatchNormalization()(u)
    u = keras.layers.Activation('relu')(u)
    u = keras.layers.Concatenate(axis=-1)([u,down_layer])
    u = keras.layers.Conv3D(k,3,padding="same",**kargs)(u)
    u = keras.layers.BatchNormalization()(u)
    u = keras.layers.Activation('relu')(u)
    u = keras.layers.Conv3D(k,3,padding="same",**kargs)(u)
    u = keras.layers.BatchNormalization()(u)
    u = keras.layers.UpSampling3D()(u)

    res = keras.layers.UpSampling3D()(previous_layer)
    res = keras.layers.Conv3D(k,1,padding="same",**kargs)(res)
    u = keras.layers.Add()([u,res])
    u = keras.layers.Activation('relu')(u)
    return u

def model(input_shape=(64,64,64,1),k1=8,reg=0.1):
    inputs=keras.layers.Input(shape=(input_shape))
    
    initializer = keras.initializers.Constant(1.)
    params = {"kernel_regularizer" : keras.regularizers.l2(reg), 
              "kernel_initializer" : 'random_normal'}
        
    d1 = down() # output should be (32,32,32,8)
    d2 = down(d1,k1*2) # output should be (16,16,16,16)
    d3 = down(d2,k1*4) # output should be (8,8,8,64)
    
    # bottleneck
    b = keras.layers.Conv3D(k1*8,3,padding="same",**params)(d3)
    b = keras.layers.BatchNormalization()(b)
    b = keras.layers.Activation('relu')(b)
    b = keras.layers.Conv3D(k1*8,3,padding="same",**params)(b)
    b = keras.layers.BatchNormalization()(b)
    b = keras.layers.Activation('relu')(b)
        
    u1 = up(b ,d3,k1*4) # output should be (8,8,8,64)
    u2 = up(u1,d2,k1*2) # output should be (16,16,16,16)
    u3 = up(u2,d1,k1) # output should be (32,32,32,8)
    
    out = keras.layers.Conv3D(,padding="same",**params)(u3) #output should be (64,64,64,1)
    out = keras.activations.sigmoid(out)
    
    model=keras.models.Model(inputs=inputs,outputs=out)
    
    return model

keras.backend.clear_session()
unet = model()
unet.summary(line_length=150)
    
    
    

It is now time to define the second most important part of a neural networ, the loss function. This is a measure of the distance between the predicted image and the target image. This function is used to compute the derivative of the network weights, so it must be differentiable. For this model we will use the dice loss, defined as $2\times\frac{A\cap B}{A\cup B}$ where A and B are respectively the volume of the reference map while B the predicted mask. The following image is a pictorical view of the same concept.


<img src="Answer/dice.png"  width="20%">

Here you should try to define 2 function, the `Dice` and `DiceLoss`. This latter one defined as 1-`Dice`. To define these functions you should use these keras functions:

* keras.backend.flatten
* keras.backend.sum

**note: try to think a way to avoid the division by zero.**

In [ ]:
def Dice(targets, inputs, smooth=1e-6):

    #flatten label and prediction tensors

    intersection = 
    dice = (2*intersection) / (a+b+smooth)
    
    return dice

def DiceLoss(targets, inputs, smooth=1e-6):
    return 1 - Dice(targets, inputs, smooth=1e-6)

Other than the loss function one has to choose which algorithm to use as an optimizer, we will use the Adam optimizer. Once this is defined the model can be compiled. We can add the dice coefficient as a metric to evaluate during the training process (both on train and validatio).

In [ ]:
adam = keras.optimizers.Adam(
    learning_rate=0.0001, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-08, 
    amsgrad=True)

unet.compile(
    loss=DiceLoss,
    optimizer=adam,
    metrics=[Dice])

While training we can change some of the values of the optimizer, in particular we can force to change the learning rate of the model accordinf to a metric. This can be performed by `keras.callbaks`methods and the metric we want to optimize is the validation dice. In the following block the variable to perform this task is define but not implemented in the fit function. try to use *google* to understand how to use this variable in fit. At the end of the fit, try to save the model as "lung_net".

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_Dice', 
    factor=0.5, patience=5, 
    min_lr=0.000005, verbose= True)

# unet = keras.models.load_model("Answer/lung_unet")

history = unet.fit(
    ,
    ,
    epochs=3,
    batch_size=1,
    validation_data=(,),
    QUI)

unet.save('lung_unet')

Now you have a trained model (if not upload the trained model from `Answers/lung_net`) try to infer (i.e. predict) the results on the test set. Try to plot the results and compare the plots with the ground trueth data.

In [ ]:
res = unet.predict(test_img)

for i,r in zip(range(len(res)),res):
    plt.title("min: {:1.4f} - max: {:1.4f}".format(r.min(),r.max()))
    plt.imshow(r[...,32,0])
    plt.colorbar()
    plt.show()
    plt.close()

other things you should try to evaluate to study how to improve the network model:

* evaluate training histrory loss and metrics
* Try to implement other metrics to evaluate the results,

## References

[1] Kamnitsas, Konstantinos, et al. "Ensembles of multiple models and architectures for robust brain tumour segmentation." International MICCAI brainlesion workshop. Springer, Cham, 2017.

[2] [Lung CT Segmentation Challenge 2017](https://wiki.cancerimagingarchive.net/display/Public/Lung+CT+Segmentation+Challenge+2017)